## POSTGRESQL 설치 방법
처음 postgresql을 사용하는 경우, Psycopg2와 postgres를 설치하여 super user 선언 및 사용자 권한 부여를 진행해야함. 다만, 주피터 노트북에서 사용하는 경우 설치 및 실행 과정이 다름.

- **Psycopg2-binary 설치**(PostgreSQL과 연결하는 Python Package)
> ```bash
> pip install psycopg2-binary
> conda install -c conda-forge postgresql
> psql --version
> ```

- **PSQL 설치** (PostgreSQL 그 자체)
> ```bash
> conda install -c conda-forge postgresql
>```
- 설치 완료 후 버전 확인
>``` bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> conda list | findstr postgresql
> postgresql                16.3                 h7f155c9_0    conda-forge
> ```
 
- **pgdata** 받아오기
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\initdb" -D "./pgdata"
> ...(log 중략)...
> Success. You can now start the database server using:
>
>   "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l logfile start
> ```
실행 이후 해당 프로젝트 내부에 *pgdata* 디렉토리가 만들어졌는지 확인할 것.

## POSTGRESQL 사용자 선언
SQL을 사용하기 위헤서는 Super User와 하부 사용자가 필요. 여기서 Super User의 이름은 *postgres*로, 하부 사용자 이름은 *d519*로 설정. 먼저, 암호 코드 설정
**1. 암호 파일 생성**
> ```bash
> echo "d519" > superuser_password.txt
> ```

**2. 오류 방지를 위해 기존의 **pgdata** 삭제 (삭제 안될 경우 파일 탐색기에서 pgdata파일 삭제)**
>```bash
> rmdir /s pgdata
> ```

**3. 새로 초기화**
> ```bash
> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> ```
- 실행 시 로그
> > ```bash
> > (branchPJH) PS C:\junha\Tilde_Chatbot> initdb -D ./pgdata --auth-local=md5 --auth-host=md5 --username=postgres --pwprompt
> > The files belonging to this database system will be owned by user "overr".
> > This user must also own the server process.
> > 
> > The database cluster will be initialized with locale "Korean_Korea.949".
> > Encoding "UHC" implied by locale is not allowed as a server-side encoding.
> > The default database encoding will be set to "UTF8" instead.
> > initdb: could not find suitable text search configuration for locale "Korean_Korea.949"
> > The default text search configuration will be set to "simple".
> > 
> > Data page checksums are disabled.
> > 
> > Enter new superuser password: 
> > Enter it again:
> > 
> > creating directory pgdata ... ok
> > creating subdirectories ... ok
> > selecting dynamic shared memory implementation ... windows
> > selecting default max_connections ... 100
> > selecting default shared_buffers ... 128MB
> > selecting default time zone ... Asia/Seoul
> > creating configuration files ... ok
> > running bootstrap script ... ok
> > performing post-bootstrap initialization ... ok
> > syncing data to disk ... ok
> > 
> > Success. You can now start the database server using:
> > 
> >     ^"C^:^\Users^\overr^\anaconda3^\envs^\branchPJH^\Library^\bin^\pg^_ctl^" -D ./pgdata -l logfile start
> > ```
   
**4. `initdb` 명령 실행**
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
- 실행 시 로그
> > ```bash
> > waiting for server to start.... done
> > server started
> > ```

# PostgreSQL 내부 작업

**1. Postgres 실행 확인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> netstat -an | Select-String "5432"
> TCP    127.0.0.1:5432         0.0.0.0:0              LISTENING
> TCP    [::1]:5432             [::]:0                 LISTENING
> ```

**2. 로그인**
> ```bash
> (branchPJH) PS C:\junha\Tilde_Chatbot> psql -U postgres -h localhost -p 5432
> Password for user postgres: 
> psql (16.3)
> Type "help" for help.
> postgres=#
> ```
`postgres=#`가 나오면 로그인 성공.

**3. User 추가**
> ```bash
> postgres=# CREATE ROLE d519 WITH LOGIN PASSWORD 'd519';
> CREATE ROLE
> ```

**4. DB 선언**
> ```bash
> postgres=# CREATE DATABASE TildeChatBotDB; 
> CREATE DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
> (4 rows)
> ```

**5. tildechatbotdb 소유주를 postgres에서 d519로 변경**
> ```bash
> postgres=# ALTER DATABASE tildechatbotdb OWNER TO d519;
> ALTER DATABASE
> postgres=# \l
>                                                                List of databases
>       Name      |  Owner   | Encoding | Locale Provider |     Collate      |      Ctype       | ICU Locale | ICU Rules |   Access privileges
> ----------------+----------+----------+-----------------+------------------+------------------+------------+-----------+-----------------------
>  postgres       | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           |
>  template0      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  template1      | postgres | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =c/postgres          +
>                 |          |          |                 |                  |                  |            |           | postgres=CTc/postgres
>  tildechatbotdb | d519     | UTF8     | libc            | Korean_Korea.949 | Korean_Korea.949 |            |           | =Tc/d519             +
>                 |          |          |                 |                  |                  |            |           | d519=CTc/d519
> (4 rows)
> ```

**6. 로그아웃**
> ```bash
> postgres=# \q
> (branchPJH) PS C:\junha\Tilde_Chatbot>
> ```


In [33]:
import pandas as pd
import psycopg2

# 전원 다시 시작시
- Terminal에서 PostgerSQL 서버 시작하기  
> ```bash
> & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start
> ```
 
- Terminal에서 super user인 **postgres**로 로그인
> ```bash
> psql -U postgres -h localhost -p 5432
> ```

In [34]:
# & "C:\Users\overr\anaconda3\envs\branchPJH\Library\bin\pg_ctl" -D "./pgdata" -l "logfile" start

In [35]:
# Postgres 연결 확인
DB_CONFIG = {
        'host': 'localhost',
        'database': 'tildechatbotdb',
        'user': 'd519',
        'password': 'd519',
        'port': 5432
    }

try:
    # PostgreSQL에 연결
    conn = psycopg2.connect(**DB_CONFIG)
    print("PostgreSQL 연결 성공!")
    conn.close()
except Exception as e:
    print(f"PostgreSQL 연결 실패: {e}")


PostgreSQL 연결 성공!


## 초기 작업 : 엑셀 내용 삽입
2024년 12월 29일 기준으로 PostgreSQL 내부 tildechatbotdb에는 ChatTable이라는 이름의 테이블이 존재. 엑셀 내용이 전부 삽입되어있는 상태임. 만약, 새로 선언한다면, 주석을 제거하고 실행할 것.

In [36]:
# def create_table_from_excel_psycopg2(excel_file, table_name, db_config):
#     df = pd.read_excel(excel_file)
# 
#     conn = psycopg2.connect(**db_config)
#     cursor = conn.cursor()
# 
#     columns = ', '.join([f'"{col}" TEXT' for col in df.columns])
#     create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})'
#     cursor.execute(create_table_query)
# 
#     for _, row in df.iterrows():
#         column_names = ', '.join([f'"{col}"' for col in df.columns])
#         placeholders = ', '.join(['%s'] * len(row))
#         insert_query = f'INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})'
#         cursor.execute(insert_query, tuple(row.values))
# 
#     conn.commit()
#     cursor.close()
#     conn.close()
#     print(f"Table '{table_name}' created and data inserted successfully.")

In [37]:
# EXCEL_FILE = "C:/junha/Datasets/ChatData.xlsx"
# TABLE_NAME = "ChatTable"
# DB_CONFIG = {
#         'host': 'localhost',
#         'database': 'tildechatbotdb',
#         'user': 'd519',
#         'password': 'd519',
#         'port': 5432
#     }
# 
# create_table_from_excel_psycopg2(EXCEL_FILE, TABLE_NAME, DB_CONFIG)


## 데이터베이스 삽입 상태 확인

In [38]:
def verify_data_insertion(table_name, db_config):
    try:
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)

        result = cursor.fetchall()

        if not result:
            print(f"No data found in table '{table_name}'.")
            return None

        column_names = [desc[0] for desc in cursor.description]

        df = pd.DataFrame(result, columns=column_names)
        print(f"Data in table '{table_name}':")

        cursor.close()
        conn.close()
        return df

    except Exception as e:
        print(f"Error verifying data: {e}")


In [39]:
verify_data_insertion('ChatTable',DB_CONFIG)

Data in table 'ChatTable':


,Data_no,구분,데이터제공,임상과,제목,질문원본,질문수정,나이,성별,노출제품유형,...,노출시간,노출후증상,몸무게,노출용량,노출후응급처치,답변원본,답변수정,답변생성1,Date,Doctor
0,217,일반인,주형준,소아청소년과,5개월아기 치약을 잘못썼어요ㅜㅜ.,5개월아기 이제 치아가 하나 나서 처음 치약쓰는거라 애기용품점에서 추천받고 보지도않...,"5개월아기 치약을 잘못썼어요ㅜㅜ., 5개월아기 이제 치아가 하나 나서 처음 치약쓰는...",5개월,NaN,생활화학제품,...,일회성,NaN,NaN,불소 250,NaN,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 엄민용 입니다. 영아의 치솔질은 ...,NaN,"5개월 아기에게 사용된 치약이 불소 250ppm 함유된 2단계 치약이라면, 소량 사...",2023.12.26.,엄민용
1,2457,의료진,중독센터,NaN,NaN,고려대학교 안암병원입니다.\n중독 물질 관련 정보 의뢰드립니다. \n\n# 노출 대...,1) 나이 및 성별 (소아는 몸무게) : F/23\n\n2) 노출 물질 및 경로(용...,23세,여자,의약품,...,NaN,drowsy,NaN,NaN,NaN,안녕하세요. 서울시 독성물질 중독관리센터입니다.\n\n요청하신 약품의 의약품안전나라...,Abilify 정 5mg 아빌리파이\nhttps://nedrug.mfds.go.kr...,NaN,NaN,상담원
2,2028,의료진,중독센터,NaN,NaN,안녕하세요 고대구로병원입니다\n약물정보의뢰드립니다\nMale 50세 \nacampr...,"Male 50세 \nacamprosate, mirtazapine, zolpid \n...",50세,남자,의약품,...,NaN,NaN,NaN,NaN,NaN,"안녕하세요, 요청하신 약물의 의약품 안전나라 약물정보 URL입니다.\n\n- aca...",acamprosate -날트렉손\nhttps://nedrug.mfds.go.kr/p...,NaN,NaN,상담원
3,1585,일반인,생성,NaN,NaN,NaN,"저희 아들이 14살인데요, 최근에 집중력 문제로 병원에서 ADHD 약을 처방받아서 ...",14세,남자,의약품,...,1개월,"두통, 심장 두근거림, 짜증, 멍하게 앉아 있음",NaN,하루 60mg,NaN,NaN,NaN,"메틸페니데이트는 ADHD 치료에 흔히 사용되는 약물로, 주의력 결핍 및 과잉행동을 ...",NaN,gpt-4o-2024-11-20
4,1724,일반인,생성,NaN,NaN,NaN,"제가 9살 남자 조카를 돌보고 있는데, 몸무게는 27kg이에요. 오늘 오후에 공부를...",9세,남자,의약품,...,30분,"가슴 두근거림, 얼굴 약간 빨개짐, 의식 명확, 호흡 문제 없음",27kg,NaN,병원 방문 권장,NaN,NaN,"메틸페니데이트는 주로 ADHD 치료에 사용되는 약물로, 중추신경계 자극제입니다. 이...",NaN,gpt-4o-2024-11-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,411,일반인,생성,NaN,NaN,NaN,"제가 29살 여자고, 오늘 공장에서 일하다가 희석제를 다루는 작업 중에 손등 쪽으로...",29세,여자,공업용화학제품,...,10초,손등 피부가 간지럽고 붉어지며 찌릿찌릿한 느낌,NaN,일부,흐르는 물로 씻어냄,NaN,NaN,"톨루엔은 피부에 접촉 시 자극을 유발할 수 있는 화학물질로, 피부를 통해 일부 흡수...",NaN,gpt-4o-2024-11-20
2518,2402,의료진,중독센터,NaN,NaN,고려대학교 안산병원 응급의료센터 중독약물 분석의뢰 드립니다.\n성별/나이 : F/3...,성별/나이 : F/35\n의도적 / 자택 / 경구\n음독약물명 : 히스티온정,35세,여자,의약품,...,NaN,NaN,NaN,NaN,NaN,안녕하세요. 서울시 독성물질 중독관리센터입니다.\n\n요청하신 약품의 의약품안전나라...,히스티온 https://nedrug.mfds.go.kr/pbp/CCBBB01/get...,NaN,NaN,상담원
2519,158,일반인,주형준,소아청소년과,아이가 유치원에서 모발검사에서 납수치가 3.9ppm이나왔어요 기준치가 2ppm이던데...,아이가 유치원에서 모발검사에서 납수치가 3.9ppm이나왔어요 기준치가 2ppm이던데...,아이가 유치원에서 모발검사에서 납수치가 3.9ppm이나왔어요 기준치가 2ppm이던데...,NaN,NaN,중금속,...,NaN,NaN,NaN,3.9ppm,NaN,안녕하세요. 하이닥-네이버 지식iN 상담의 한재병 입니다. ​ 체내 흡수된 납을 배...,NaN,모발검사를 통해 확인된 납 수치가 기준치(2ppm)를 초과한 3.9ppm으로 나왔다...,2023.11.28.,한재병
2520,162,일반인,주형준,응급의학과,비스페놀a (BPA)로 인한 화상,일하다가 비스페놀a(BPA) 화학물질이안면부에 튀어 화상을 입었습니다 어제 급히 화...,"비스페놀a (BPA)로 인한 화상, 일하다가 비스페놀a(BPA) 화학물질이안면부에 ...",NaN,NaN,공업용화학제품,...,일회성,화상,NaN,NaN,화상전문병원에서 응급처치를 받음,화학화상입니다. 물집이 생기는 2도화상 이하의 화상이라면 흉터 걱정은 안하셔도됩니...,NaN,"비스페놀 A(BPA)는 주로 플라스틱 제조에 사용되는 화학물질로, 폴리카보네이트 플...",2021.05.17.,김태완


## 원하는 값만 추출

In [41]:
def select_distinct_values(query, db_config):
    try:
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        
        distinct_values = [row[0] for row in results]
        return distinct_values
    except Exception as e:
        print("Database error:", e)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [62]:
TBL_NAME = "ChatTable"
COLUMN_NAME = "노출제품명"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
distinct_values = select_distinct_values(QUERY, DB_CONFIG)
distinct_values

Database error: relation "information_schema" does not exist
LINE 1: SELECT column_name FROM information_schema WHERE table_name ...
                                ^


In [43]:
def fetch_matching_rows(distinct_values, table_name, column_name, db_config):
    try:
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()

        all_results = []

        for value in distinct_values:
            query = f"SELECT * FROM {table_name} WHERE {column_name} LIKE '%{value}%';"
            cursor.execute(query)
            rows = cursor.fetchall()

            # 결과 저장
            all_results.extend(rows)
        
        return all_results
    except Exception as e:
        print("Database error:", e)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [46]:
TBL_NAME = "ChatTable"
COLUMN_NAME = "노출제품명"
QUERY = f"SELECT DISTINCT {COLUMN_NAME} FROM {TBL_NAME};"
distinct_values = select_distinct_values(QUERY, DB_CONFIG)

question = "이 제품은 농약이 포함되어 있나요? 무농약 제품은 어떻습니까?"

matching_rows = fetch_matching_rows(distinct_values, TBL_NAME, COLUMN_NAME, DB_CONFIG)
matching_rows

[('332',
  '일반인',
  '생성',
  'NaN',
  'NaN',
  'NaN',
  '저희 할아버지께서 올해 72세 되셨는데, 최근 갑자기 안 좋아지신 것 같아서요. 어떤 이유인가 봤더니 작은 이벤트장에서 화학 작업이 있었고, 할아버지가 30분 정도 노란색 연기를 마셨던 것 같아요. 정확히 무슨 연기였는지는 모르겠고, 라텔락 과정 중 나왔던 유독가스라고 합니다. 이 노출 이후로 어지러움이 심하고, 숨 쉬기가 불편하시다고 하세요. 가슴이 답답하시다는데, 큰 병원으로 가야 할까요? 이런 경우 응급으로 처리해야 할 후속 조치가 있다면 알려주세요. 추가로, 할아버지가 오랫동안 기역양을 치료하기 약하며 혈기성이 약한 건 아닐까요? 감사합니다.  ',
  '72세',
  '남자',
  '유해가스',
  '유독가스',
  '라텔락 과정 중 발생한 노란색 연기',
  '호흡기',
  '작은 이벤트장에서 30분 동안 노란색 연기를 흡입',
  '30분',
  '어지러움, 숨 쉬기 불편, 가슴 답답함',
  'NaN',
  'NaN',
  '큰 병원으로 이동하여 전문적인 진단 및 치료 필요',
  'NaN',
  'NaN',
  '할아버지께서 노란색 연기를 흡입한 후 어지러움, 호흡 곤란, 가슴 답답함 등의 증상을 호소하신다면, 이는 유독가스 노출로 인한 급성 호흡기 및 전신 반응일 가능성이 높습니다. 특히 라텔락 과정에서 발생한 유독가스라면, 이는 화학물질로 인해 발생한 것으로 보이며, 정확한 성분을 알 수 없으므로 신속한 의료 조치가 필요합니다.\n\n우선, 할아버지께서 현재 호흡 곤란이나 가슴 답답함을 지속적으로 느끼신다면, 즉시 응급실로 이동하셔야 합니다. 유독가스 노출은 폐와 호흡기에 심각한 손상을 줄 수 있으며, 심한 경우 산소 부족, 폐부종, 또는 중추신경계 손상으로 이어질 수 있습니다. 응급실에서는 산소 치료, 혈액 검사, 흉부 X-ray, 또는 폐 기능 검사를 통해 상태를 평가하고 적절한 치료를 제공할 수 있습니다.\n\n응급 조치로는 할아버지를 

In [63]:
columns = ['Data_no', '구분', '데이터제공', '제목', '질문원본', '질문수정', '나이', '성별', '노출제품유형', '노출제품명', '노출성분명', '노출부위', '노출내용', '노출시간', '노출후증상', '몸무게', '노출용량', '노출후응급처치', '답변생성1', 'Date', 'Doctor']
df = pd.DataFrame(matching_rows, columns=columns)
print(df)

ValueError: 21 columns passed, passed data had 24 columns